In [1]:
from cartoframes.auth import Credentials, set_default_credentials

credentials = Credentials.from_file()
set_default_credentials(credentials)

1. Read and understand the data

In [2]:
import pandas
from cartoframes import CartoDataFrame

df = pandas.read_csv('../files/starbucks_brooklyn.csv')
df.head()

,name,address,revenue
0,Franklin Ave & Eastern Pkwy,"341 Eastern Pkwy,Brooklyn, NY 11238",1321040.772
1,607 Brighton Beach Ave,"607 Brighton Beach Avenue,Brooklyn, NY 11235",1268080.418
2,65th St & 18th Ave,"6423 18th Avenue,Brooklyn, NY 11204",1248133.699
3,Bay Ridge Pkwy & 3rd Ave,"7419 3rd Avenue,Brooklyn, NY 11209",1185702.676
4,Caesar's Bay Shopping Center,"8973 Bay Parkway,Brooklyn, NY 11214",1148427.411


2. Before using data services: check the quota

In [3]:
from cartoframes.data.services import Geocoding

geo_service = Geocoding()

_, geo_dry_metadata = geo_service.geocode(
    df,
    street='address',
    city={'value': 'New York'},
    country={'value': 'USA'},
    dry_run=True
)

Debug: creating table "table_d499a39c13"
Success! Data uploaded correctly


In [4]:
geo_dry_metadata

{'total_rows': 10,
 'required_quota': 10,
 'previously_geocoded': 0,
 'previously_failed': 0,
 'records_with_geometry': 0}

Geocode but storing the results using the `cached` option, and that the dataset created is `private` by default

In [8]:
geo_cdf, geo_metadata = geo_service.geocode(
    df,
    street='address',
    city={'value': 'New York'},
    country={'value': 'USA'},
    table_name='starbucks_cache',
    cached=True
)

Debug: creating table "starbucks_cache"
Success! Data uploaded correctly
Success! Data geocoded correctly


Compare previous metadata with current metadata

In [9]:
geo_metadata

{'total_rows': 10,
 'required_quota': 10,
 'previously_geocoded': 0,
 'previously_failed': 0,
 'records_with_geometry': 0,
 'final_records_with_geometry': 10,
 'geocoded_increment': 10,
 'successfully_geocoded': 10,
 'failed_geocodings': 0}

Check `gc_status_rel`, `carto_geocode_hash` and `geometry` new columns

In [10]:
geo_cdf.head()

,name,address,revenue,gc_status_rel,carto_geocode_hash,geometry
1,Franklin Ave & Eastern Pkwy,"341 Eastern Pkwy,Brooklyn, NY 11238",1321040.772,0.97,c834a8e289e5bce280775a9bf1f833f1,POINT (-73.95901 40.67109)
2,607 Brighton Beach Ave,"607 Brighton Beach Avenue,Brooklyn, NY 11235",1268080.418,0.99,7d39a3fff93efd9034da88aa9ad2da79,POINT (-73.96122 40.57796)
3,65th St & 18th Ave,"6423 18th Avenue,Brooklyn, NY 11204",1248133.699,0.98,1a2312049ddea753ba42bf77f5ccf718,POINT (-73.98976 40.61912)
4,Bay Ridge Pkwy & 3rd Ave,"7419 3rd Avenue,Brooklyn, NY 11209",1185702.676,0.98,827ab4dcc2d49d5fd830749597976d4a,POINT (-74.02744 40.63152)
5,Caesar's Bay Shopping Center,"8973 Bay Parkway,Brooklyn, NY 11214",1148427.411,0.98,119a38c7b51195cd4153fc81605a8495,POINT (-74.00098 40.59321)


In [11]:
geo_cdf.gc_status_rel.unique()

array([0.97, 0.99, 0.98])

In [12]:
from cartoframes.viz.helpers import color_bins_layer
from cartoframes.viz import Popup

color_bins_layer(
    geo_cdf,
    'gc_status_rel',
    method='equal',
    bins=geo_cdf.gc_status_rel.unique().size,
    title='Geocoding Precision',
    popup=Popup({
        'hover': [{
                'title': 'Address',
                'value': '$address'
            }, {
                'title': 'Precision',
                'value': '$gc_status_rel'
            }]
    })
)

Check available quota

In [13]:
geo_service.available_quota()

4995617

In [15]:
from cartoframes.data.services import Isolines

iso_service = Isolines()

_, isochrones_dry_metadata = iso_service.isochrones(geo_cdf, [450, 900, 1800], mode='walk', dry_run=True)

In [16]:
print('available {0}, required {1}'.format(iso_service.available_quota(), isochrones_dry_metadata.get('required_quota')))

available 110507, required 30


In [17]:
isochrones_cdf, isochrones_metadata = iso_service.isochrones(geo_cdf, [450, 900, 1800], mode='walk')

Debug: creating table "table_66d82506fd"
Success! Data uploaded correctly
Debug: table "table_66d82506fd" removed
Success! Isolines correctly created


In [18]:
isochrones_cdf.head()

,data_range,geometry
1,450,"MULTIPOLYGON (((-73.95933 40.67377, -73.95993 ..."
2,900,"MULTIPOLYGON (((-73.95934 40.68011, -73.96074 ..."
3,1800,"MULTIPOLYGON (((-73.95949 40.69066, -73.96353 ..."
4,450,"MULTIPOLYGON (((-73.96212 40.58244, -73.96273 ..."
5,900,"MULTIPOLYGON (((-73.96187 40.58632, -73.96289 ..."


In [19]:
from cartoframes.viz.helpers import color_bins_layer

color_bins_layer(isochrones_cdf, 'data_range', bins=3)

In [20]:
isolines_layer(isochrones_cdf)

KeyError: "['range_label'] not in index"

Explain other settings available, using isodistances this time:

In [ ]:
isodistances_cdf, isodistances_dry_metadata = iso_service.isodistances(
    geo_cdf,
    [900, 1800, 3600],
    mode='walk',
    resolution=16.0,
    quality=1,
    dry_run=True
)

In [ ]:
isodistances_dry_metadata.get('required_quota')

In [ ]:
isodistances_cdf, isodistances_metadata = iso_service.isodistances(
    geo_cdf,
    [900, 1800, 3600],
    mode='walk',
    mode_traffic='enabled',
    resolution=16.0,
    quality=2
)

In [ ]:
isodistances_cdf.head()

In [ ]:
isodistances_cdf = isodistances_cdf[isodistances_cdf['geometry'].is_empty == False]

Remove empty geometries:

In [7]:
from cartoframes.viz.helpers import isolines_layer

isolines_layer(isodistances_cdf)

NameError: name 'isolines_layer' is not defined

All together

In [ ]:
from cartoframes.viz import Map

Map([
    isolines_layer(isodistances_cdf),
    color_bins_layer(
        geo_cdf,
        'gc_status_rel',
        method='equal',
        bins=geo_cdf.gc_status_rel.unique().size,
        title='Geocoding Precision',
        popup=Popup({
            'hover': [{
                    'title': 'Address',
                    'value': '$address'
                }, {
                    'title': 'Precision',
                    'value': '$gc_status_rel'
                }]
        })
    )
])